#### Import necessary libraries

In [1]:
import botocore
import boto3
import pprint

from langchain_aws import ChatBedrock
from langchain_aws import BedrockEmbeddings
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever

from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

#### Verify the ID for the existing Knowledge Base in Amazon Bedrock:

In [2]:
session = boto3.Session()
bedrock_client = session.client('bedrock-agent')

try:
    response = bedrock_client.list_knowledge_bases(
        maxResults=1  
    )
    knowledge_base_summaries = response.get('knowledgeBaseSummaries', [])

    if knowledge_base_summaries:
        kb_id = knowledge_base_summaries[0]['knowledgeBaseId']
        print(f"Knowledge Base ID: {kb_id}")
    else:
        print("No Knowledge Base summaries found.")
        
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

Knowledge Base ID: SK5WRI3ZBN


#### Select the models that will be consumed by the knowledgebase

In [3]:
pp = pprint.PrettyPrinter(indent=2)

bedrock_client = boto3.client('bedrock-runtime')
llm_for_text_generation = ChatBedrock(model_id="amazon.nova-micro-v1:0", client=bedrock_client)
llm_for_evaluation = ChatBedrock(model_id="amazon.nova-lite-v1:0", client=bedrock_client)

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0",client=bedrock_client)

#### Create a *AmazonKnowledgeBasesRetriever* object using langchain

In [4]:
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 5}},
        region_name="us-east-1",
    )
retriever

C:\Users\tochi\AppData\Local\Temp\ipykernel_25616\1171070506.py:1: LangChainDeprecationWarning: The class `AmazonKnowledgeBasesRetriever` was deprecated in LangChain 0.3.16 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import AmazonKnowledgeBasesRetriever``.
  retriever = AmazonKnowledgeBasesRetriever(


AmazonKnowledgeBasesRetriever(knowledge_base_id='SK5WRI3ZBN', region_name='us-east-1', client=<botocore.client.AgentsforBedrockRuntime object at 0x00000298BE2F73D0>, retrieval_config=RetrievalConfig(vectorSearchConfiguration=VectorSearchConfig(numberOfResults=5)))

4. Run the following code cell to create a prompt with context and question as variables:

In [5]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are an AI system that assists with standard operating procedures (SOPs) for opioid treatment in the field of psychiatry. Your answers must be based exclusively on the provided ASAM National Practice Guideline for the Treatment of Opioid Use Disorder documents.
Use the following pieces of information to provide a concise and professional answer to the question enclosed in <question> tags. 
If you don't know the answer based on the provided documents, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use clinical guidance, statistics, or numbers when possible, as found in the guidelines.

Assistant:"""
prompt = PromptTemplate(template=PROMPT_TEMPLATE,
                              input_variables=["context","question"])

5. Run the following code cell to invoke the model using a pre-defined query and print the results:

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs): #concatenate the text from the page_content field in the output from retriever.invoke
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm_for_text_generation
    | StrOutputParser()
)


query = "What are the three primary medications for Opioid Use Disorder and what is the ASAM definition of addiction"

response=chain.invoke(query)
print(response)

According to the ASAM National Practice Guideline for the Treatment of Opioid Use Disorder, the three primary medications used for the treatment of Opioid Use Disorder (OUD) are:

1. **Methadone**: Methadone is a long-acting opioid agonist that helps to alleviate withdrawal symptoms and reduce cravings. It is typically administered in a controlled clinical setting and is often part of a comprehensive treatment program that includes counseling and other support services.

2. **Buprenorphine**: Buprenorphine is a partial opioid agonist that can be prescribed in both office-based settings and through specialized opioid treatment programs. It is effective in reducing withdrawal symptoms and cravings, and it has a lower risk of misuse and overdose compared to other opioids.

3. **Naltrexone**: Naltrexone is an opioid antagonist that blocks the euphoric effects of opioids. It is available in both oral and extended-release injectable forms. Naltrexone is often used after detoxification to hel

#### Prepare the *question* and *ground_truths* pairs for evaluation:

In [7]:
from datasets import Dataset
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# --- Prompt Template for Guardrails and Persona ---
prompt_template_string = """
Human: You are an internal AI assistant for a multi-state outpatient psychiatric group. Your primary purpose is to provide accurate information from the provided Standard Operating Procedures (SOPs), policies, and internal documentation. The documents are the ASAM National Practice Guideline for the Treatment of Opioid Use Disorder.

**Strictly adhere to the following rules:**
1.  **ONLY** answer questions using the information provided in the given context.
2.  If the answer to the question is not explicitly found in the provided context, state clearly: 'I cannot find an answer to that question in the provided documents.' **DO NOT** use your general knowledge or make up information.
3.  For every piece of information you provide, cite the document name and relevant section/page number from the source.
4.  **DO NOT** provide any medical advice, diagnoses, or treatment recommendations.
5.  **DO NOT** discuss any topics unrelated to the clinic's internal operations, policies, or SOPs. If asked an unrelated question, respond with: 'My purpose is to provide information from our internal clinic documents. I am unable to answer questions outside this scope.'
6.  Maintain a professional and helpful tone.

Context: {context}

Question: {question}

Answer:
"""
PROMPT = PromptTemplate(
    template=prompt_template_string,
    input_variables=["context", "question"]
)

# --- Create the RAG Chain using RetrievalQA ---
chain = RetrievalQA.from_chain_type(
    llm=llm_for_text_generation,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT} 
)

# --- The test data ---
questions = [
    "What are the three primary medications for Opioid Use Disorder mentioned in the guideline?",
    "What is the ASAM definition of addiction?",
    "What is the recommended approach for individuals with opioid use disorder who are in the criminal justice system?",
    "Is it necessary to complete a full assessment before initiating pharmacotherapy for opioid use disorder?",
    "When can naloxone be administered?",
    "What are the primary goals of the AHRQ Surveys on Patient Safety Culture (SOPS) program?",
    "Does the guideline specify a time limit for pharmacological treatment?",
    "What should be done when a new communication technology is introduced in a healthcare setting concerning HIPAA compliance?",
    "Name at least three psychiatric disorders for which the APA provides specific clinical practice guidelines."
]

ground_truth = [
    "The three primary medications mentioned for treating Opioid Use Disorder are methadone, buprenorphine, and naltrexone. (Source: ASAM National Practice Guideline, various sections)",
    "According to the ASAM, addiction is a 'treatable, chronic medical disease involving complex interactions among brain circuits, genetics, the environment, and an individual’s life experiences'. It is characterized by compulsive substance use or behaviors that continue despite harmful consequences. (Source: ASAM National Practice Guideline, Section 2)",
    "Patients with opioid use disorder who are in prison or jail should be stabilized on pharmacotherapy (methadone, buprenorphine, or naltrexone) and continue treatment after their release. Patient care upon reentry to the community should be individualized and coordinated with community treatment providers. Additionally, individuals should receive a naloxone kit and education on how to use it before being released. (Source: ASAM National Practice Guideline, Section 8)",
    "While a comprehensive assessment is critical, its completion should not delay or prevent the initiation of pharmacotherapy. The assessment should be completed soon after treatment begins if it cannot be done beforehand. (Source: ASAM OPIOID TREATMENT GUIDELINE (POCKET GUIDE), page 1)",
    "Naloxone should be administered in the event of a suspected opioid overdose. It may also be administered to pregnant women in overdose cases to save the mother's life. (Source: ASAM National Practice Guideline, Section 7)",
    "I cannot find an answer to that question in the provided documents.",
    "No, there is no recommended time limit for pharmacological treatment. (Source: ASAM OPIOID TREATMENT GUIDELINE (POCKET GUIDE), page 2)",
    "I cannot find an answer to that question in the provided documents.",
    "I cannot find an answer to that question in the provided documents."
]

answers = []
contexts = []

for query in questions:
    response = chain.invoke({"query": query})
    answers.append(response["result"])
    contexts.append([doc.page_content for doc in response.get("source_documents", [])])

data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truth
}

dataset = Dataset.from_dict(data)

print("\n--- Evaluation Dataset Generated ---")
print(dataset[0])


--- Evaluation Dataset Generated ---
{'question': 'What are the three primary medications for Opioid Use Disorder mentioned in the guideline?', 'answer': 'The three primary medications for Opioid Use Disorder mentioned in the guideline are:\n\n1. Methadone\n2. Buprenorphine\n3. Naltrexone\n\n(Source: The ASAM National Practice Guideline for the Treatment of Opioid Use Disorder, Parts 4–6)', 'contexts': ['(See Appendix III for an overview of the main pharmacotherapy options and Appendix IV for a summary of available formulations). The FDA    � 2020 American Society of Addiction Medicine    approvals for these medications have primarily been for ‘opioid dependence’ as defined in prior versions of the DSM, and not necessarily the definition contained in the current version of the manual, the DSM-5. DSM-5 combined opioid abuse and opioid dependence criteria from prior versions of the DSM and included them in the new definition of opioid use disorder. As a result, pharmacologic treatment m

#### See the answers from the LLM and the ground truths for the evaluation set of questions:

In [10]:
i=0
for answer in answers:
    i=i+1
    print(str(i)+').'+questions[i-1]+'\n')
    print("LLM:" +answer+'\n')
    print ("Ground truth: "+ ground_truth[i-1]+'\n')

1).What are the three primary medications for Opioid Use Disorder mentioned in the guideline?

LLM:The three primary medications for Opioid Use Disorder mentioned in the guideline are:

1. Methadone
2. Buprenorphine
3. Naltrexone

(Source: The ASAM National Practice Guideline for the Treatment of Opioid Use Disorder, Parts 4–6)

Ground truth: The three primary medications mentioned for treating Opioid Use Disorder are methadone, buprenorphine, and naltrexone. (Source: ASAM National Practice Guideline, various sections)

2).What is the ASAM definition of addiction?

LLM:The ASAM definition of addiction is: "Addiction is a treatable, chronic medical disease involving complex interactions among brain circuits, genetics, the environment, and an individual’s life experiences. People with addiction use substances or engage in behaviors that become compulsive and often continue despite harmful consequences." [Source: ASAM National Practice Guideline for the Treatment of Opioid Use Disorder, B

#### Import all the metrics from *ragas.metrics* and use the *evaluate()* function:

In [11]:
import warnings
warnings.filterwarnings('ignore')  

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_similarity,
    answer_correctness
)

from ragas.metrics.critique import (
harmfulness, 
maliciousness, 
coherence, 
correctness, 
conciseness
)

#specify the metrics here
metrics = [
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        context_entity_recall,
        answer_similarity,
        answer_correctness,
        harmfulness, 
        maliciousness, 
        coherence, 
        correctness, 
        conciseness
    ]

result = evaluate(
    dataset = dataset, 
    metrics=metrics,
    llm=llm_for_evaluation,
    embeddings=bedrock_embeddings,
)

df = result.to_pandas()
result

Evaluating:   0%|          | 0/108 [00:00<?, ?it/s]

{'faithfulness': 0.5833, 'answer_relevancy': 0.4583, 'context_precision': 0.4849, 'context_recall': 0.5000, 'context_entity_recall': 0.2065, 'answer_similarity': 0.6844, 'answer_correctness': 0.5623, 'harmfulness': 0.0000, 'maliciousness': 0.1111, 'coherence': 1.0000, 'correctness': 1.0000, 'conciseness': 1.0000}

#### Run the below code cell to see the resulting detailed RAGAS scores:

In [12]:
import pandas as pd
pd.options.display.max_colwidth = 10
df.style.set_sticky(axis="columns")

#### Export the resulting RAGAS scores in excel format:

In [48]:
df.style.to_excel('styled.xlsx', engine='openpyxl')